re
bs4
xpath 推荐

In [ ]:
import re

ls = re.findall(r"\d+", "123345")
print(ls)
# iter效率高
it = re.finditer(r"\d+", "123我45")
for i in it:
    print(i.group())
# re.search匹配一个结果
# re.match从头开始匹配

#预加载正则表达式
obj = re.compile(r"\d+")
it = obj.finditer("124 1254")
print(it)

In [ ]:
import re

s = """
<div class='jay'><span id='1'>周杰伦</span></div>
<div class='kk'><span id='2'>zzq</span></div>
"""

# re.S 让.能匹配换行符
# obj = re.compile(r"<div class='.*?'><span id='\d+'>.*?</span></div>", re.S)
# result = obj.finditer(s)
# for i in result:
#     print(i.group())

# (?P<分组名字>正则表达式)从正则里面提取
obj = re.compile(r"<div class='.*?'><span id='\d+'>(?P<name>.*?)</span></div>", re.S)
result = obj.finditer(s)
for i in result:
    print(i.group('name'))

In [ ]:
import requests
import re
import csv

url = 'https://movie.douban.com/top250'

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
}

resp = requests.get(url, headers=headers)
page_content = resp.text
resp.close()

obj = re.compile(r'<li>.*?<div class="item">.*?<span class="title">(?P<title>.*?)</span>'
                 r'.*?<p class="">.*?<br>(?P<year>.*?)&nbsp.*?'
                 r'<span class="rating_num" property="v:average">(?P<score>.*?)</span>'
                 r'.*?<span>(?P<num>.*?)人评价</span>', re.S)

results = obj.finditer(page_content)

f = open('data.csv', 'w')
cw = csv.writer(f)

for it in results:
    dic = it.groupdict()
    dic['year'] = dic['year'].strip()
    cw.writerow(dic.values())
    print(it.group("title"), end=' ')
    print(it.group("year").strip(), end=' ')
    print(it.group("score"), end=' ')
    print(it.group("num"))

f.close()


In [ ]:
# install bs4

import requests
from bs4 import BeautifulSoup

url = 'http://www.xinfadi.com.cn/marketanalysis/0/list/1.shtml'
resp = requests.get(url)
# print(resp.text)

# 解析数据
page = BeautifulSoup(resp.text, 'html.parser')
# 查找数据 find一个 findall全部，获取总表
table = page.find("table", class_="hq_table")
# print(table) 获取每一行
trs = table.find_all("tr")[1:]

for tr in trs:
    tds = tr.find_all("td")
    print(tds)
    for td in tds:
        print(td.text)

resp.close()

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

url = 'https://www.umei.net/bizhitupian/weimeibizhi/'

resp = requests.get(url)
resp.encoding = 'utd-8'

page = BeautifulSoup(resp.text, 'html.parser')
TypeList = page.find("div", class_='TypeList').find_all("a")

for a in TypeList:
    href = 'https://www.umei.net/' + a.get('href')
    child_page_resp = requests.get(href)
    child_page_resp.encoding = 'utd-8'
    child_page = BeautifulSoup(child_page_resp.text, 'html.parser')
    img = child_page.find("p", align="center").find('img')
    print(img.get('src'))
    src = img.get('src')
    #下载图片
    img_resp = requests.get(src)
    img_name = src.split("/")[-1]

    with open('img/'+img_name, mode='wb') as f:
        f.write(img_resp.content)
    child_page_resp.close()
    time.sleep(1)

resp.close()

xpath 是在xml文档中搜索内容的一门语言
html 是xml的一个子集
pip install lxml

https://www.jianshu.com/p/85a3004b5c06

表达式	描述
nodename	选取此节点的所有子节点
/	从当前节点选区直接子节点
//	从当前节点选取子孙节点
.	选取当前节点
..	选取当前节点的父节点
@	选取属性
这里列出了 XPath 的常用匹配规则，示例如下：

//title[@lang='eng']
这是一个 XPath 规则，代表的是选择所有名称为 title，同时属性 lang 的值为 eng 的节点，后面会通过 Python 的 lxml 库，利用 XPath 进行 HTML 的解析。

In [ ]:
from lxml import etree

xml = """
<book>
    <name>123</name>
    <author>456</author>
</book>
"""

#etree.HTML
tree = etree.XML(xml)

result = tree.xpath('/book')
print(result)

result = tree.xpath('/book/name')
print(result)

result = tree.xpath('/book/name/text()')
print(result)

In [ ]:
import requests
from lxml import etree

url = "https://ganzhou.zbj.com/search/f/?type=new&kw=saas"

resp = requests.get(url)

html = etree.HTML(resp.text)

divs = html.xpath("/html/body/div[6]/div/div/div[2]/div[5]/div[1]/div")

for div in divs:
    try:
        price = div.xpath("./div/div/a[1]/div[2]/div[1]/span[1]/text()")[0].strip("¥")
        title = 'saas'.join(div.xpath("./div/div/a[1]/div[2]/div[2]/p/text()"))
        print(price, title)
    except:
        continue

